In [44]:
import json
from pandas.io.json import json_normalize
import requests
import pyspark as ps
import pandas as pd
import numpy as np
spark = (ps.sql.SparkSession.builder 
        .master("local[4]") 
        .appName("sparkSQL exercise") 
        .getOrCreate()
        )
sc = spark.sparkContext

In [113]:
# Methods
def unpack_column(dataFrame,col_name):
    newDf = dataFrame.from_records(dataFrame[col_name].dropna().tolist())
    return newDf

In [66]:
xf = pd.read_json("https://api.data.charitynavigator.org/v2/Organizations?app_id=4239274b&app_key=c78b1e5cf16a02eaf43992fbb60ce979&pageSize=100&pageNum=1&rated=true&sort=RATING%3ADESC")

In [67]:
jf = xf.copy()

In [95]:
#jf = jf.drop('currentRating', axis = 1)
jf = jf.drop('advisories', axis = 1)

In [111]:
jf.head(1)

,category,cause,charityName,charityNavigatorURL,donationAddress,ein,irsClassification,mailingAddress,mission,organization,tagLine,websiteURL,rating,categoryID,categoryName
0,"{'categoryName': 'Health', 'categoryID': 5, 'c...","{'causeID': 13, 'causeName': 'Diseases, Disord...",Florida Breast Cancer Foundation,https://www.charitynavigator.org/?bay=search.s...,NaN,10694045,{'deductibility': 'Contributions are deductibl...,"{'country': None, 'stateOrProvince': 'FL', 'ci...",Florida Breast Cancer Foundation strives to en...,{'charityName': 'Florida Breast Cancer Foundat...,To end the suffering caused by breast cancer,http://www.floridabreastcancer.org/,4,5,Health


In [108]:
#cat = unpack_column(jf,"category")
#cat.head()

In [109]:
# jf['categoryID'] = cat['categoryID']
# jf['categoryName'] = cat['categoryName']

In [114]:
cause = unpack_column(jf,"cause")
cause.head()

,causeID,causeName,charityNavigatorURL,image
0,13,"Diseases, Disorders, and Disciplines",https://www.charitynavigator.org/index.cfm?bay...,https://d20umu42aunjpx.cloudfront.net/_gfx_/ca...
1,20,Development and Relief Services,https://www.charitynavigator.org/index.cfm?bay...,https://d20umu42aunjpx.cloudfront.net/_gfx_/ca...
2,2,"Animal Rights, Welfare, and Services",https://www.charitynavigator.org/index.cfm?bay...,https://d20umu42aunjpx.cloudfront.net/_gfx_/ca...
3,11,Environmental Protection and Conservation,https://www.charitynavigator.org/index.cfm?bay...,https://d20umu42aunjpx.cloudfront.net/_gfx_/ca...
4,42,United Ways,https://www.charitynavigator.org/index.cfm?bay...,https://d20umu42aunjpx.cloudfront.net/_gfx_/ca...


In [126]:
# jf['causeID'] = cause['causeID']
# jf['causeName'] = cause['causeName']
# jf = jf.drop('category',axis=1)
# jf = jf.drop('cause',axis=1)
# jf = jf.drop('charityNavigatorURL',axis=1)
#jf = jf.drop(columns=['donationAddress','websiteURL'])
jf.head()

,charityName,ein,irsClassification,mailingAddress,mission,organization,tagLine,rating,categoryID,categoryName,causeID,causeName
0,Florida Breast Cancer Foundation,10694045,{'deductibility': 'Contributions are deductibl...,"{'country': None, 'stateOrProvince': 'FL', 'ci...",Florida Breast Cancer Foundation strives to en...,{'charityName': 'Florida Breast Cancer Foundat...,To end the suffering caused by breast cancer,4,5,Health,13,"Diseases, Disorders, and Disciplines"
1,Haitian Health Foundation,61135999,{'deductibility': 'Contributions are deductibl...,"{'country': None, 'stateOrProvince': 'CT', 'ci...",The mission of the Haitian Health Foundation (...,"{'charityName': 'Haitian Health Foundation', '...",Changing Lives ~ One Person at a Time,4,7,International,20,Development and Relief Services
2,SPCA of Westchester,131740069,{'deductibility': 'Contributions are deductibl...,"{'country': None, 'stateOrProvince': 'NY', 'ci...",The SPCA of Westchester was founded in 1883. T...,"{'charityName': 'SPCA of Westchester', 'ein': ...",Over 100 years of service to animals and their...,4,1,Animals,2,"Animal Rights, Welfare, and Services"
3,Conservation Foundation of the Gulf Coast,200345249,{'deductibility': 'Contributions are deductibl...,"{'country': None, 'stateOrProvince': 'FL', 'ci...",Conservation Foundation of the Gulf Coast work...,{'charityName': 'Conservation Foundation of th...,Conservation through education and collaboration,4,4,Environment,11,Environmental Protection and Conservation
4,United Way of Blount County,237122193,{'deductibility': 'Contributions are deductibl...,"{'country': None, 'stateOrProvince': 'TN', 'ci...",United Way of Blount County is advancing the c...,"{'charityName': 'United Way of Blount County',...",Live united,4,10,Community Development,42,United Ways


In [127]:
def search_and_destroy(dataFrame,column):
    temp = unpack_column(dataFrame,column)
    for col in temp.columns:
        dataFrame[col] = temp[col]
    return dataFrame

In [128]:
x = jf.copy()

In [129]:
search_and_destroy(x,"irsClassification")

,charityName,ein,irsClassification,mailingAddress,mission,organization,tagLine,rating,categoryID,categoryName,...,groupName,incomeAmount,latest990,nteeClassification,nteeCode,nteeLetter,nteeSuffix,nteeType,rulingDate,subsection
0,Florida Breast Cancer Foundation,10694045,{'deductibility': 'Contributions are deductibl...,"{'country': None, 'stateOrProvince': 'FL', 'ci...",Florida Breast Cancer Foundation strives to en...,{'charityName': 'Florida Breast Cancer Foundat...,To end the suffering caused by breast cancer,4,5,Health,...,None,1662411,"June, 2018",Fund Raising and/or Fund Distribution,H12,H,None,Medical Research,"April, 2003",501(c)(3)
1,Haitian Health Foundation,61135999,{'deductibility': 'Contributions are deductibl...,"{'country': None, 'stateOrProvince': 'CT', 'ci...",The mission of the Haitian Health Foundation (...,"{'charityName': 'Haitian Health Foundation', '...",Changing Lives ~ One Person at a Time,4,7,International,...,None,15631712,"July, 2017","International Development, Relief Services",Q30,Q,0,"International, Foreign Affairs and National Se...","December, 1985",501(c)(3)
2,SPCA of Westchester,131740069,{'deductibility': 'Contributions are deductibl...,"{'country': None, 'stateOrProvince': 'NY', 'ci...",The SPCA of Westchester was founded in 1883. T...,"{'charityName': 'SPCA of Westchester', 'ein': ...",Over 100 years of service to animals and their...,4,1,Animals,...,None,5363562,"December, 2017",Animal Protection and Welfare,D20,D,0,Animal-Related,"December, 1959",501(c)(3)
3,Conservation Foundation of the Gulf Coast,200345249,{'deductibility': 'Contributions are deductibl...,"{'country': None, 'stateOrProvince': 'FL', 'ci...",Conservation Foundation of the Gulf Coast work...,{'charityName': 'Conservation Foundation of th...,Conservation through education and collaboration,4,4,Environment,...,None,5269608,"September, 2018",Natural Resources Conservation and Protection,C30,C,None,"Environmental Quality, Protection and Beautifi...","December, 2003",501(c)(3)
4,United Way of Blount County,237122193,{'deductibility': 'Contributions are deductibl...,"{'country': None, 'stateOrProvince': 'TN', 'ci...",United Way of Blount County is advancing the c...,"{'charityName': 'United Way of Blount County',...",Live united,4,10,Community Development,...,None,2279080,"December, 2018",Fund Raising Organizations That Cross Categories,T70,T,0,"Philanthropy, Voluntarism and Grantmaking Foun...","August, 1971",501(c)(3)
5,Hope for Haiti's Children,311811917,{'deductibility': 'Contributions are deductibl...,"{'country': None, 'stateOrProvince': 'OH', 'ci...",Hope For Haiti's Children (HFHC) is a faith-ba...,"{'charityName': 'Hope for Haiti's Children', '...",Making a Difference...One Child at a Time,4,7,International,...,None,3409996,"December, 2018",Fund Raising and/or Fund Distribution,G12,G,8,"Diseases, Disorders, Medical Disciplines","August, 2001",501(c)(3)
6,The Rotary Foundation of Rotary International,363245072,{'deductibility': 'Contributions are deductibl...,"{'country': None, 'stateOrProvince': 'IL', 'ci...",The Rotary Foundation is a not-for-profit corp...,{'charityName': 'The Rotary Foundation of Rota...,Enabling Rotarians to advance world understand...,4,7,International,...,None,3362234653,"June, 2018",Single Organization Support,Q11,Q,3,"International, Foreign Affairs and National Se...","August, 1983",501(c)(3)
7,Lifesong for Orphans,351902841,{'deductibility': 'Contributions are deductibl...,"{'country': None, 'stateOrProvince': 'IL', 'ci...",Lifesong for Orphans seeks to mobilize the Chu...,"{'charityName': 'Lifesong for Orphans', 'ein':...",Bringing joy and purpose to orphans,4,6,Human Services,...,None,21924369,"June, 2018",International Relief,Q33,Q,0,"International, Foreign Affairs and National Se...","March, 1994",501(c)(3)
8,Environmental and Energy Study Institute,521268030,{'deductibility': 'Contributions are deductibl...,"{'country': None, 'stateOrProvince': 'DC', 'ci...",EESI is dedicated to promoti

In [130]:
x = x.drop(columns=['nteeCode','nteeLetter','nteeSuffix'])

In [134]:
# x = x.drop(columns=['filingRequirement','foundationStatus','groupName'])

In [136]:
x = x.drop(columns=['deductibilityDetail','exemptOrgStatusCode','latest990'])
x

,charityName,ein,irsClassification,mailingAddress,mission,organization,tagLine,rating,categoryID,categoryName,...,assetAmount,classification,deductibility,deductibilityCode,exemptOrgStatus,incomeAmount,nteeClassification,nteeType,rulingDate,subsection
0,Florida Breast Cancer Foundation,10694045,{'deductibility': 'Contributions are deductibl...,"{'country': None, 'stateOrProvince': 'FL', 'ci...",Florida Breast Cancer Foundation strives to en...,{'charityName': 'Florida Breast Cancer Foundat...,To end the suffering caused by breast cancer,4,5,Health,...,2221944,Charitable Organization,Contributions are deductible,1,Unconditional Exemption,1662411,Fund Raising and/or Fund Distribution,Medical Research,"April, 2003",501(c)(3)
1,Haitian Health Foundation,61135999,{'deductibility': 'Contributions are deductibl...,"{'country': None, 'stateOrProvince': 'CT', 'ci...",The mission of the Haitian Health Foundation (...,"{'charityName': 'Haitian Health Foundation', '...",Changing Lives ~ One Person at a Time,4,7,International,...,46434088,Charitable Organization,Contributions are deductible,1,Unconditional Exemption,15631712,"International Development, Relief Services","International, Foreign Affairs and National Se...","December, 1985",501(c)(3)
2,SPCA of Westchester,131740069,{'deductibility': 'Contributions are deductibl...,"{'country': None, 'stateOrProvince': 'NY', 'ci...",The SPCA of Westchester was founded in 1883. T...,"{'charityName': 'SPCA of Westchester', 'ein': ...",Over 100 years of service to animals and their...,4,1,Animals,...,6612191,Charitable Organization,Contributions are deductible,1,Unconditional Exemption,5363562,Animal Protection and Welfare,Animal-Related,"December, 1959",501(c)(3)
3,Conservation Foundation of the Gulf Coast,200345249,{'deductibility': 'Contributions are deductibl...,"{'country': None, 'stateOrProvince': 'FL', 'ci...",Conservation Foundation of the Gulf Coast work...,{'charityName': 'Conservation Foundation of th...,Conservation through education and collaboration,4,4,Environment,...,30114667,Charitable Organization,Contributions are deductible,1,Unconditional Exemption,5269608,Natural Resources Conservation and Protection,"Environmental Quality, Protection and Beautifi...","December, 2003",501(c)(3)
4,United Way of Blount County,237122193,{'deductibility': 'Contributions are deductibl...,"{'country': None, 'stateOrProvince': 'TN', 'ci...",United Way of Blount County is advancing the c...,"{'charityName': 'United Way of Blount County',...",Live united,4,10,Community Development,...,4358002,Charitable Organization,Contributions are deductible,1,Unconditional Exemption,2279080,Fund Raising Organizations That Cross Categories,"Philanthropy, Voluntarism and Grantmaking Foun...","August, 1971",501(c)(3)
5,Hope for Haiti's Children,311811917,{'deductibility': 'Contributions are deductibl...,"{'country': None, 'stateOrProvince': 'OH', 'ci...",Hope For Haiti's Children (HFHC) is a faith-ba...,"{'charityName': 'Hope for Haiti's Children', '...",Making a Difference...One Child at a Time,4,7,International,...,5047175,Charitable Organization,Contributions are deductible,1,Unconditional Exemption,3409996,Fund Raising and/or Fund Distribution,"Diseases, Disorders, Medical Disciplines","August, 2001",501(c)(3)
6,The Rotary Foundation of Rotary International,363245072,{'deductibility': 'Contributions are deductibl...,"{'country': None, 'stateOrProvince': 'IL', 'ci...",The Rotary Foundation is a not-for-profit corp...,{'charityName': 'The Rotary Foundation of Rota...,Enabling Rotarians to advance world understand...,4,7,International,...,1254692074,Charitable Organization,Contributions are deductible,1,Unconditional Exemption,3362234653,Single Organization Support,"International, Foreign Affairs and National Se...","August, 1983",501(c)(3)
7,Lifesong for Orphans,351902841,{'deductibility': 'Contributions are deductibl...,"{'country': None, 'stateOrProvince': 'IL', 'ci...",Lifesong for Orphans seeks to mobilize the 

In [137]:
x = x.drop(columns=['subsection','rulingDate','exemptOrgStatus'])
x

,charityName,ein,irsClassification,mailingAddress,mission,organization,tagLine,rating,categoryID,categoryName,...,causeName,accountingPeriod,affiliation,assetAmount,classification,deductibility,deductibilityCode,incomeAmount,nteeClassification,nteeType
0,Florida Breast Cancer Foundation,10694045,{'deductibility': 'Contributions are deductibl...,"{'country': None, 'stateOrProvince': 'FL', 'ci...",Florida Breast Cancer Foundation strives to en...,{'charityName': 'Florida Breast Cancer Foundat...,To end the suffering caused by breast cancer,4,5,Health,...,"Diseases, Disorders, and Disciplines",June,Independent - the organization is an independe...,2221944,Charitable Organization,Contributions are deductible,1,1662411,Fund Raising and/or Fund Distribution,Medical Research
1,Haitian Health Foundation,61135999,{'deductibility': 'Contributions are deductibl...,"{'country': None, 'stateOrProvince': 'CT', 'ci...",The mission of the Haitian Health Foundation (...,"{'charityName': 'Haitian Health Foundation', '...",Changing Lives ~ One Person at a Time,4,7,International,...,Development and Relief Services,July,Independent - the organization is an independe...,46434088,Charitable Organization,Contributions are deductible,1,15631712,"International Development, Relief Services","International, Foreign Affairs and National Se..."
2,SPCA of Westchester,131740069,{'deductibility': 'Contributions are deductibl...,"{'country': None, 'stateOrProvince': 'NY', 'ci...",The SPCA of Westchester was founded in 1883. T...,"{'charityName': 'SPCA of Westchester', 'ein': ...",Over 100 years of service to animals and their...,4,1,Animals,...,"Animal Rights, Welfare, and Services",December,Independent - the organization is an independe...,6612191,Charitable Organization,Contributions are deductible,1,5363562,Animal Protection and Welfare,Animal-Related
3,Conservation Foundation of the Gulf Coast,200345249,{'deductibility': 'Contributions are deductibl...,"{'country': None, 'stateOrProvince': 'FL', 'ci...",Conservation Foundation of the Gulf Coast work...,{'charityName': 'Conservation Foundation of th...,Conservation through education and collaboration,4,4,Environment,...,Environmental Protection and Conservation,September,Independent - the organization is an independe...,30114667,Charitable Organization,Contributions are deductible,1,5269608,Natural Resources Conservation and Protection,"Environmental Quality, Protection and Beautifi..."
4,United Way of Blount County,237122193,{'deductibility': 'Contributions are deductibl...,"{'country': None, 'stateOrProvince': 'TN', 'ci...",United Way of Blount County is advancing the c...,"{'charityName': 'United Way of Blount County',...",Live united,4,10,Community Development,...,United Ways,December,Independent - the organization is an independe...,4358002,Charitable Organization,Contributions are deductible,1,2279080,Fund Raising Organizations That Cross Categories,"Philanthropy, Voluntarism and Grantmaking Foun..."
5,Hope for Haiti's Children,311811917,{'deductibility': 'Contributions are deductibl...,"{'country': None, 'stateOrProvince': 'OH', 'ci...",Hope For Haiti's Children (HFHC) is a faith-ba...,"{'charityName': 'Hope for Haiti's Children', '...",Making a Difference...One Child at a Time,4,7,International,...,Development and Relief Services,December,Independent - the organization is an independe...,5047175,Charitable Organization,Contributions are deductible,1,3409996,Fund Raising and/or Fund Distribution,"Diseases, Disorders, Medical Disciplines"
6,The Rotary Foundation of Rotary International,363245072,{'deductibility': 'Contributions are deductibl...,"{'country': None, 'stateOrProvince': 'IL', 'ci...",The Rotary Foundation is a not-for-profit corp...,{'charityName': 'The Rotary Foundation of Rota...,Enabling Rotarians to advance world understand...,4,7,International,...,"International Peace, Security, and Affairs",June,Independent - the organization is an independe...,1254692074,Charitable Organization,Cont

In [138]:
search_and_destroy(x,"mailingAddress")

,charityName,ein,irsClassification,mailingAddress,mission,organization,tagLine,rating,categoryID,categoryName,...,deductibilityCode,incomeAmount,nteeClassification,nteeType,city,country,postalCode,stateOrProvince,streetAddress1,streetAddress2
0,Florida Breast Cancer Foundation,10694045,{'deductibility': 'Contributions are deductibl...,"{'country': None, 'stateOrProvince': 'FL', 'ci...",Florida Breast Cancer Foundation strives to en...,{'charityName': 'Florida Breast Cancer Foundat...,To end the suffering caused by breast cancer,4,5,Health,...,1,1662411,Fund Raising and/or Fund Distribution,Medical Research,North Miami,None,33181,FL,11900 Biscayne Boulevard,Suite 288
1,Haitian Health Foundation,61135999,{'deductibility': 'Contributions are deductibl...,"{'country': None, 'stateOrProvince': 'CT', 'ci...",The mission of the Haitian Health Foundation (...,"{'charityName': 'Haitian Health Foundation', '...",Changing Lives ~ One Person at a Time,4,7,International,...,1,15631712,"International Development, Relief Services","International, Foreign Affairs and National Se...",Norwich,None,06360,CT,97 Sherman Street,None
2,SPCA of Westchester,131740069,{'deductibility': 'Contributions are deductibl...,"{'country': None, 'stateOrProvince': 'NY', 'ci...",The SPCA of Westchester was founded in 1883. T...,"{'charityName': 'SPCA of Westchester', 'ein': ...",Over 100 years of service to animals and their...,4,1,Animals,...,1,5363562,Animal Protection and Welfare,Animal-Related,Briarcliff Manor,None,10510,NY,590 North State Road,None
3,Conservation Foundation of the Gulf Coast,200345249,{'deductibility': 'Contributions are deductibl...,"{'country': None, 'stateOrProvince': 'FL', 'ci...",Conservation Foundation of the Gulf Coast work...,{'charityName': 'Conservation Foundation of th...,Conservation through education and collaboration,4,4,Environment,...,1,5269608,Natural Resources Conservation and Protection,"Environmental Quality, Protection and Beautifi...",Osprey,None,34229,FL,P.O. Box 902,400 Palmetto Avenue
4,United Way of Blount County,237122193,{'deductibility': 'Contributions are deductibl...,"{'country': None, 'stateOrProvince': 'TN', 'ci...",United Way of Blount County is advancing the c...,"{'charityName': 'United Way of Blount County',...",Live united,4,10,Community Development,...,1,2279080,Fund Raising Organizations That Cross Categories,"Philanthropy, Voluntarism and Grantmaking Foun...",Maryville,None,37804,TN,1615 E. Broadway Avenue,None
5,Hope for Haiti's Children,311811917,{'deductibility': 'Contributions are deductibl...,"{'country': None, 'stateOrProvince': 'OH', 'ci...",Hope For Haiti's Children (HFHC) is a faith-ba...,"{'charityName': 'Hope for Haiti's Children', '...",Making a Difference...One Child at a Time,4,7,International,...,1,3409996,Fund Raising and/or Fund Distribution,"Diseases, Disorders, Medical Disciplines",Cincinnati,None,45241,OH,12020 Southwick Lane,None
6,The Rotary Foundation of Rotary International,363245072,{'deductibility': 'Contributions are deductibl...,"{'country': None, 'stateOrProvince': 'IL', 'ci...",The Rotary Foundation is a not-for-profit corp...,{'charityName': 'The Rotary Foundation of Rota...,Enabling Rotarians to advance world understand...,4,7,International,...,1,3362234653,Single Organization Support,"International, Foreign Affairs and National Se...",Evanston,None,60201,IL,One Rotary Center,1560 Sherman Avenue
7,Lifesong for Orphans,351902841,{'deductibility': 'Contributions are deductibl...,"{'country': None, 'stateOrProvince': 'IL', 'ci...",Lifesong for Orphans seeks to mobilize the Chu...,"{'charityName': 'Lifesong for Orphans', 'ein':...",Bringing joy and purpose to orphans,4,6,Human Services,...,1,21924369,International Relief,"International, Foreign Affairs and National Se...",Gridley,None,61744,IL,P. O. Box 40,202 North Ford Street
8,Environmental and Energy Study Institute,521268030,{'deductibility': 'Contributions are deductibl...,"{'country': None, 'stateOrProvince': 'DC', '

In [140]:
x = x.drop(columns=['irsClassification','mailingAddress','country','streetAddress2'])
x

,charityName,ein,mission,organization,tagLine,rating,categoryID,categoryName,causeID,causeName,...,classification,deductibility,deductibilityCode,incomeAmount,nteeClassification,nteeType,city,postalCode,stateOrProvince,streetAddress1
0,Florida Breast Cancer Foundation,10694045,Florida Breast Cancer Foundation strives to en...,{'charityName': 'Florida Breast Cancer Foundat...,To end the suffering caused by breast cancer,4,5,Health,13,"Diseases, Disorders, and Disciplines",...,Charitable Organization,Contributions are deductible,1,1662411,Fund Raising and/or Fund Distribution,Medical Research,North Miami,33181,FL,11900 Biscayne Boulevard
1,Haitian Health Foundation,61135999,The mission of the Haitian Health Foundation (...,"{'charityName': 'Haitian Health Foundation', '...",Changing Lives ~ One Person at a Time,4,7,International,20,Development and Relief Services,...,Charitable Organization,Contributions are deductible,1,15631712,"International Development, Relief Services","International, Foreign Affairs and National Se...",Norwich,06360,CT,97 Sherman Street
2,SPCA of Westchester,131740069,The SPCA of Westchester was founded in 1883. T...,"{'charityName': 'SPCA of Westchester', 'ein': ...",Over 100 years of service to animals and their...,4,1,Animals,2,"Animal Rights, Welfare, and Services",...,Charitable Organization,Contributions are deductible,1,5363562,Animal Protection and Welfare,Animal-Related,Briarcliff Manor,10510,NY,590 North State Road
3,Conservation Foundation of the Gulf Coast,200345249,Conservation Foundation of the Gulf Coast work...,{'charityName': 'Conservation Foundation of th...,Conservation through education and collaboration,4,4,Environment,11,Environmental Protection and Conservation,...,Charitable Organization,Contributions are deductible,1,5269608,Natural Resources Conservation and Protection,"Environmental Quality, Protection and Beautifi...",Osprey,34229,FL,P.O. Box 902
4,United Way of Blount County,237122193,United Way of Blount County is advancing the c...,"{'charityName': 'United Way of Blount County',...",Live united,4,10,Community Development,42,United Ways,...,Charitable Organization,Contributions are deductible,1,2279080,Fund Raising Organizations That Cross Categories,"Philanthropy, Voluntarism and Grantmaking Foun...",Maryville,37804,TN,1615 E. Broadway Avenue
5,Hope for Haiti's Children,311811917,Hope For Haiti's Children (HFHC) is a faith-ba...,"{'charityName': 'Hope for Haiti's Children', '...",Making a Difference...One Child at a Time,4,7,International,20,Development and Relief Services,...,Charitable Organization,Contributions are deductible,1,3409996,Fund Raising and/or Fund Distribution,"Diseases, Disorders, Medical Disciplines",Cincinnati,45241,OH,12020 Southwick Lane
6,The Rotary Foundation of Rotary International,363245072,The Rotary Foundation is a not-for-profit corp...,{'charityName': 'The Rotary Foundation of Rota...,Enabling Rotarians to advance world understand...,4,7,International,19,"International Peace, Security, and Affairs",...,Charitable Organization,Contributions are deductible,1,3362234653,Single Organization Support,"International, Foreign Affairs and National Se...",Evanston,60201,IL,One Rotary Center
7,Lifesong for Orphans,351902841,Lifesong for Orphans seeks to mobilize the Chu...,"{'charityName': 'Lifesong for Orphans', 'ein':...",Bringing joy and purpose to orphans,4,6,Human Services,17,Children's and Family Services,...,Charitable Organization,Contributions are deductible,1,21924369,International Relief,"International, Foreign Affairs and National Se...",Gridley,61744,IL,P. O. Box 40
8,Environmental and Energy Study Institute,521268030,EESI is dedicated to promoting environmentally...,{'charityName': 'Environmental and Energy Stud...,Ideas. Insights. Sustainable Solutions.,4,4,Environment,11,Environmental Protection and Conservation,...,Educational Organization,Contributions are deductible,1,2245148,None,None,Washington,20036,DC,1020 19th St. NW
9,United Way of C

In [141]:
search_and_destroy(x,"organization")

,charityName,ein,mission,organization,tagLine,rating,categoryID,categoryName,causeID,causeName,...,deductibilityCode,incomeAmount,nteeClassification,nteeType,city,postalCode,stateOrProvince,streetAddress1,_rapid_links,charityNavigatorURL
0,Florida Breast Cancer Foundation,010694045,Florida Breast Cancer Foundation strives to en...,{'charityName': 'Florida Breast Cancer Foundat...,To end the suffering caused by breast cancer,4,5,Health,13,"Diseases, Disorders, and Disciplines",...,1,1662411,Fund Raising and/or Fund Distribution,Medical Research,North Miami,33181,FL,11900 Biscayne Boulevard,{'related': {'href': 'https://api.data.charity...,https://www.charitynavigator.org/?bay=search.s...
1,Haitian Health Foundation,061135999,The mission of the Haitian Health Foundation (...,"{'charityName': 'Haitian Health Foundation', '...",Changing Lives ~ One Person at a Time,4,7,International,20,Development and Relief Services,...,1,15631712,"International Development, Relief Services","International, Foreign Affairs and National Se...",Norwich,06360,CT,97 Sherman Street,{'related': {'href': 'https://api.data.charity...,https://www.charitynavigator.org/?bay=search.s...
2,SPCA of Westchester,131740069,The SPCA of Westchester was founded in 1883. T...,"{'charityName': 'SPCA of Westchester', 'ein': ...",Over 100 years of service to animals and their...,4,1,Animals,2,"Animal Rights, Welfare, and Services",...,1,5363562,Animal Protection and Welfare,Animal-Related,Briarcliff Manor,10510,NY,590 North State Road,{'related': {'href': 'https://api.data.charity...,https://www.charitynavigator.org/?bay=search.s...
3,Conservation Foundation of the Gulf Coast,200345249,Conservation Foundation of the Gulf Coast work...,{'charityName': 'Conservation Foundation of th...,Conservation through education and collaboration,4,4,Environment,11,Environmental Protection and Conservation,...,1,5269608,Natural Resources Conservation and Protection,"Environmental Quality, Protection and Beautifi...",Osprey,34229,FL,P.O. Box 902,{'related': {'href': 'https://api.data.charity...,https://www.charitynavigator.org/?bay=search.s...
4,United Way of Blount County,237122193,United Way of Blount County is advancing the c...,"{'charityName': 'United Way of Blount County',...",Live united,4,10,Community Development,42,United Ways,...,1,2279080,Fund Raising Organizations That Cross Categories,"Philanthropy, Voluntarism and Grantmaking Foun...",Maryville,37804,TN,1615 E. Broadway Avenue,{'related': {'href': 'https://api.data.charity...,https://www.charitynavigator.org/?bay=search.s...
5,Hope for Haiti's Children,311811917,Hope For Haiti's Children (HFHC) is a faith-ba...,"{'charityName': 'Hope for Haiti's Children', '...",Making a Difference...One Child at a Time,4,7,International,20,Development and Relief Services,...,1,3409996,Fund Raising and/or Fund Distribution,"Diseases, Disorders, Medical Disciplines",Cincinnati,45241,OH,12020 Southwick Lane,{'related': {'href': 'https://api.data.charity...,https://www.charitynavigator.org/?bay=search.s...
6,The Rotary Foundation of Rotary International,363245072,The Rotary Foundation is a not-for-profit corp...,{'charityName': 'The Rotary Foundation of Rota...,Enabling Rotarians to advance world understand...,4,7,International,19,"International Peace, Security, and Affairs",...,1,3362234653,Single Organization Support,"International, Foreign Affairs and National Se...",Evanston,60201,IL,One Rotary Center,{'related': {'href': 'https://api.data.charity...,https://www.charitynavigator.org/?bay=search.s...
7,Lifesong for Orphans,351902841,Lifesong for Orphans seeks to mobilize the Chu...,"{'charityName': 'Lifesong for Orphans', 'ein':...",Bringing joy and purpose to orphans,4,6,Human Services,17,Children's and Family Services,...,1,21924369,International Relief,"International, Foreign Affairs and National Se...",Gridley,61744,IL,P. O. Box 40,{'related': {'href': 'https://api.data.charity...,https://www.charitynavigator.org/?bay=search.s...
8,En

In [142]:
x = x.drop(columns=['organization','_rapid_links','charityNavigatorURL'])
x

,charityName,ein,mission,tagLine,rating,categoryID,categoryName,causeID,causeName,accountingPeriod,...,classification,deductibility,deductibilityCode,incomeAmount,nteeClassification,nteeType,city,postalCode,stateOrProvince,streetAddress1
0,Florida Breast Cancer Foundation,010694045,Florida Breast Cancer Foundation strives to en...,To end the suffering caused by breast cancer,4,5,Health,13,"Diseases, Disorders, and Disciplines",June,...,Charitable Organization,Contributions are deductible,1,1662411,Fund Raising and/or Fund Distribution,Medical Research,North Miami,33181,FL,11900 Biscayne Boulevard
1,Haitian Health Foundation,061135999,The mission of the Haitian Health Foundation (...,Changing Lives ~ One Person at a Time,4,7,International,20,Development and Relief Services,July,...,Charitable Organization,Contributions are deductible,1,15631712,"International Development, Relief Services","International, Foreign Affairs and National Se...",Norwich,06360,CT,97 Sherman Street
2,SPCA of Westchester,131740069,The SPCA of Westchester was founded in 1883. T...,Over 100 years of service to animals and their...,4,1,Animals,2,"Animal Rights, Welfare, and Services",December,...,Charitable Organization,Contributions are deductible,1,5363562,Animal Protection and Welfare,Animal-Related,Briarcliff Manor,10510,NY,590 North State Road
3,Conservation Foundation of the Gulf Coast,200345249,Conservation Foundation of the Gulf Coast work...,Conservation through education and collaboration,4,4,Environment,11,Environmental Protection and Conservation,September,...,Charitable Organization,Contributions are deductible,1,5269608,Natural Resources Conservation and Protection,"Environmental Quality, Protection and Beautifi...",Osprey,34229,FL,P.O. Box 902
4,United Way of Blount County,237122193,United Way of Blount County is advancing the c...,Live united,4,10,Community Development,42,United Ways,December,...,Charitable Organization,Contributions are deductible,1,2279080,Fund Raising Organizations That Cross Categories,"Philanthropy, Voluntarism and Grantmaking Foun...",Maryville,37804,TN,1615 E. Broadway Avenue
5,Hope for Haiti's Children,311811917,Hope For Haiti's Children (HFHC) is a faith-ba...,Making a Difference...One Child at a Time,4,7,International,20,Development and Relief Services,December,...,Charitable Organization,Contributions are deductible,1,3409996,Fund Raising and/or Fund Distribution,"Diseases, Disorders, Medical Disciplines",Cincinnati,45241,OH,12020 Southwick Lane
6,The Rotary Foundation of Rotary International,363245072,The Rotary Foundation is a not-for-profit corp...,Enabling Rotarians to advance world understand...,4,7,International,19,"International Peace, Security, and Affairs",June,...,Charitable Organization,Contributions are deductible,1,3362234653,Single Organization Support,"International, Foreign Affairs and National Se...",Evanston,60201,IL,One Rotary Center
7,Lifesong for Orphans,351902841,Lifesong for Orphans seeks to mobilize the Chu...,Bringing joy and purpose to orphans,4,6,Human Services,17,Children's and Family Services,June,...,Charitable Organization,Contributions are deductible,1,21924369,International Relief,"International, Foreign Affairs and National Se...",Gridley,61744,IL,P. O. Box 40
8,Environmental and Energy Study Institute,521268030,EESI is dedicated to promoting environmentally...,Ideas. Insights. Sustainable Solutions.,4,4,Environment,11,Environmental Protection and Conservation,December,...,Educational Organization,Contributions are deductible,1,2245148,None,None,Washington,20036,DC,1020 19th St. NW
9,United Way of Central Alabama,630288846,"Organized in 1923, United Way of Central Alaba...",Live United,4,10,Community Development,42,United Ways,December,...,Charitable Organization,Contributions are deductible,1,75057848,Fund Raising Organizations That Cross Categories,"Philanthropy, Voluntarism and Grantmaking Foun...",Birmingham,35232,AL,"3600 Eighth Avenue, S"


In [143]:
x = x.drop(columns=['accountingPeriod','classification'])
x

,charityName,ein,mission,tagLine,rating,categoryID,categoryName,causeID,causeName,affiliation,assetAmount,deductibility,deductibilityCode,incomeAmount,nteeClassification,nteeType,city,postalCode,stateOrProvince,streetAddress1
0,Florida Breast Cancer Foundation,010694045,Florida Breast Cancer Foundation strives to en...,To end the suffering caused by breast cancer,4,5,Health,13,"Diseases, Disorders, and Disciplines",Independent - the organization is an independe...,2221944,Contributions are deductible,1,1662411,Fund Raising and/or Fund Distribution,Medical Research,North Miami,33181,FL,11900 Biscayne Boulevard
1,Haitian Health Foundation,061135999,The mission of the Haitian Health Foundation (...,Changing Lives ~ One Person at a Time,4,7,International,20,Development and Relief Services,Independent - the organization is an independe...,46434088,Contributions are deductible,1,15631712,"International Development, Relief Services","International, Foreign Affairs and National Se...",Norwich,06360,CT,97 Sherman Street
2,SPCA of Westchester,131740069,The SPCA of Westchester was founded in 1883. T...,Over 100 years of service to animals and their...,4,1,Animals,2,"Animal Rights, Welfare, and Services",Independent - the organization is an independe...,6612191,Contributions are deductible,1,5363562,Animal Protection and Welfare,Animal-Related,Briarcliff Manor,10510,NY,590 North State Road
3,Conservation Foundation of the Gulf Coast,200345249,Conservation Foundation of the Gulf Coast work...,Conservation through education and collaboration,4,4,Environment,11,Environmental Protection and Conservation,Independent - the organization is an independe...,30114667,Contributions are deductible,1,5269608,Natural Resources Conservation and Protection,"Environmental Quality, Protection and Beautifi...",Osprey,34229,FL,P.O. Box 902
4,United Way of Blount County,237122193,United Way of Blount County is advancing the c...,Live united,4,10,Community Development,42,United Ways,Independent - the organization is an independe...,4358002,Contributions are deductible,1,2279080,Fund Raising Organizations That Cross Categories,"Philanthropy, Voluntarism and Grantmaking Foun...",Maryville,37804,TN,1615 E. Broadway Avenue
5,Hope for Haiti's Children,311811917,Hope For Haiti's Children (HFHC) is a faith-ba...,Making a Difference...One Child at a Time,4,7,International,20,Development and Relief Services,Independent - the organization is an independe...,5047175,Contributions are deductible,1,3409996,Fund Raising and/or Fund Distribution,"Diseases, Disorders, Medical Disciplines",Cincinnati,45241,OH,12020 Southwick Lane
6,The Rotary Foundation of Rotary International,363245072,The Rotary Foundation is a not-for-profit corp...,Enabling Rotarians to advance world understand...,4,7,International,19,"International Peace, Security, and Affairs",Independent - the organization is an independe...,1254692074,Contributions are deductible,1,3362234653,Single Organization Support,"International, Foreign Affairs and National Se...",Evanston,60201,IL,One Rotary Center
7,Lifesong for Orphans,351902841,Lifesong for Orphans seeks to mobilize the Chu...,Bringing joy and purpose to orphans,4,6,Human Services,17,Children's and Family Services,Independent - the organization is an independe...,23060820,Contributions are deductible,1,21924369,International Relief,"International, Foreign Affairs and National Se...",Gridley,61744,IL,P. O. Box 40
8,Environmental and Energy Study Institute,521268030,EESI is dedicated to promoting environmentally...,Ideas. Insights. Sustainable Solutions.,4,4,Environment,11,Environmental Protection and Conservation,Independent - the organization is an independe...,3816789,Contributions are deductible,1,2245148,None,None,Washington,20036,DC,1020 19th St. NW
9,United Way of Central Alabama,630288846,"Organized in 1923, United Way of Central Alaba...",Live United,4,10,Community Development,42,United Ways,Independent - the organization is an independe...,96864346,C

In [144]:
x = x.drop(columns=['affiliation'])
x

,charityName,ein,mission,tagLine,rating,categoryID,categoryName,causeID,causeName,assetAmount,deductibility,deductibilityCode,incomeAmount,nteeClassification,nteeType,city,postalCode,stateOrProvince,streetAddress1
0,Florida Breast Cancer Foundation,010694045,Florida Breast Cancer Foundation strives to en...,To end the suffering caused by breast cancer,4,5,Health,13,"Diseases, Disorders, and Disciplines",2221944,Contributions are deductible,1,1662411,Fund Raising and/or Fund Distribution,Medical Research,North Miami,33181,FL,11900 Biscayne Boulevard
1,Haitian Health Foundation,061135999,The mission of the Haitian Health Foundation (...,Changing Lives ~ One Person at a Time,4,7,International,20,Development and Relief Services,46434088,Contributions are deductible,1,15631712,"International Development, Relief Services","International, Foreign Affairs and National Se...",Norwich,06360,CT,97 Sherman Street
2,SPCA of Westchester,131740069,The SPCA of Westchester was founded in 1883. T...,Over 100 years of service to animals and their...,4,1,Animals,2,"Animal Rights, Welfare, and Services",6612191,Contributions are deductible,1,5363562,Animal Protection and Welfare,Animal-Related,Briarcliff Manor,10510,NY,590 North State Road
3,Conservation Foundation of the Gulf Coast,200345249,Conservation Foundation of the Gulf Coast work...,Conservation through education and collaboration,4,4,Environment,11,Environmental Protection and Conservation,30114667,Contributions are deductible,1,5269608,Natural Resources Conservation and Protection,"Environmental Quality, Protection and Beautifi...",Osprey,34229,FL,P.O. Box 902
4,United Way of Blount County,237122193,United Way of Blount County is advancing the c...,Live united,4,10,Community Development,42,United Ways,4358002,Contributions are deductible,1,2279080,Fund Raising Organizations That Cross Categories,"Philanthropy, Voluntarism and Grantmaking Foun...",Maryville,37804,TN,1615 E. Broadway Avenue
5,Hope for Haiti's Children,311811917,Hope For Haiti's Children (HFHC) is a faith-ba...,Making a Difference...One Child at a Time,4,7,International,20,Development and Relief Services,5047175,Contributions are deductible,1,3409996,Fund Raising and/or Fund Distribution,"Diseases, Disorders, Medical Disciplines",Cincinnati,45241,OH,12020 Southwick Lane
6,The Rotary Foundation of Rotary International,363245072,The Rotary Foundation is a not-for-profit corp...,Enabling Rotarians to advance world understand...,4,7,International,19,"International Peace, Security, and Affairs",1254692074,Contributions are deductible,1,3362234653,Single Organization Support,"International, Foreign Affairs and National Se...",Evanston,60201,IL,One Rotary Center
7,Lifesong for Orphans,351902841,Lifesong for Orphans seeks to mobilize the Chu...,Bringing joy and purpose to orphans,4,6,Human Services,17,Children's and Family Services,23060820,Contributions are deductible,1,21924369,International Relief,"International, Foreign Affairs and National Se...",Gridley,61744,IL,P. O. Box 40
8,Environmental and Energy Study Institute,521268030,EESI is dedicated to promoting environmentally...,Ideas. Insights. Sustainable Solutions.,4,4,Environment,11,Environmental Protection and Conservation,3816789,Contributions are deductible,1,2245148,None,None,Washington,20036,DC,1020 19th St. NW
9,United Way of Central Alabama,630288846,"Organized in 1923, United Way of Central Alaba...",Live United,4,10,Community Development,42,United Ways,96864346,Contributions are deductible,1,75057848,Fund Raising Organizations That Cross Categories,"Philanthropy, Voluntarism and Grantmaking Foun...",Birmingham,35232,AL,"3600 Eighth Avenue, S"


In [145]:
search_and_destroy(x,"ein")

,charityName,ein,mission,tagLine,rating,categoryID,categoryName,causeID,causeName,assetAmount,...,streetAddress1,0,1,2,3,4,5,6,7,8
0,Florida Breast Cancer Foundation,010694045,Florida Breast Cancer Foundation strives to en...,To end the suffering caused by breast cancer,4,5,Health,13,"Diseases, Disorders, and Disciplines",2221944,...,11900 Biscayne Boulevard,0,1,0,6,9,4,0,4,5
1,Haitian Health Foundation,061135999,The mission of the Haitian Health Foundation (...,Changing Lives ~ One Person at a Time,4,7,International,20,Development and Relief Services,46434088,...,97 Sherman Street,0,6,1,1,3,5,9,9,9
2,SPCA of Westchester,131740069,The SPCA of Westchester was founded in 1883. T...,Over 100 years of service to animals and their...,4,1,Animals,2,"Animal Rights, Welfare, and Services",6612191,...,590 North State Road,1,3,1,7,4,0,0,6,9
3,Conservation Foundation of the Gulf Coast,200345249,Conservation Foundation of the Gulf Coast work...,Conservation through education and collaboration,4,4,Environment,11,Environmental Protection and Conservation,30114667,...,P.O. Box 902,2,0,0,3,4,5,2,4,9
4,United Way of Blount County,237122193,United Way of Blount County is advancing the c...,Live united,4,10,Community Development,42,United Ways,4358002,...,1615 E. Broadway Avenue,2,3,7,1,2,2,1,9,3
5,Hope for Haiti's Children,311811917,Hope For Haiti's Children (HFHC) is a faith-ba...,Making a Difference...One Child at a Time,4,7,International,20,Development and Relief Services,5047175,...,12020 Southwick Lane,3,1,1,8,1,1,9,1,7
6,The Rotary Foundation of Rotary International,363245072,The Rotary Foundation is a not-for-profit corp...,Enabling Rotarians to advance world understand...,4,7,International,19,"International Peace, Security, and Affairs",1254692074,...,One Rotary Center,3,6,3,2,4,5,0,7,2
7,Lifesong for Orphans,351902841,Lifesong for Orphans seeks to mobilize the Chu...,Bringing joy and purpose to orphans,4,6,Human Services,17,Children's and Family Services,23060820,...,P. O. Box 40,3,5,1,9,0,2,8,4,1
8,Environmental and Energy Study Institute,521268030,EESI is dedicated to promoting environmentally...,Ideas. Insights. Sustainable Solutions.,4,4,Environment,11,Environmental Protection and Conservation,3816789,...,1020 19th St. NW,5,2,1,2,6,8,0,3,0
9,United Way of Central Alabama,630288846,"Organized in 1923, United Way of Central Alaba...",Live United,4,10,Community Development,42,United Ways,96864346,...,"3600 Eighth Avenue, S",6,3,0,2,8,8,8,4,6


In [149]:
type(x['ein'][0])

str

In [151]:
type(jf['irsClassification'][0])

dict

In [154]:
search_and_destroy(xf,'ratingImage')

,advisories,category,cause,charityName,charityNavigatorURL,currentRating,donationAddress,ein,irsClassification,mailingAddress,mission,organization,tagLine,websiteURL,rating,ratingImage,large,small
0,"{'severity': None, 'active': {'_rapid_links': ...","{'categoryName': 'Health', 'categoryID': 5, 'c...","{'causeID': 13, 'causeName': 'Diseases, Disord...",Florida Breast Cancer Foundation,https://www.charitynavigator.org/?bay=search.s...,{'ratingImage': {'small': 'https://d20umu42aun...,NaN,10694045,{'deductibility': 'Contributions are deductibl...,"{'country': None, 'stateOrProvince': 'FL', 'ci...",Florida Breast Cancer Foundation strives to en...,{'charityName': 'Florida Breast Cancer Foundat...,To end the suffering caused by breast cancer,http://www.floridabreastcancer.org/,4,{'small': 'https://d20umu42aunjpx.cloudfront.n...,https://d20umu42aunjpx.cloudfront.net/_gfx_/ic...,https://d20umu42aunjpx.cloudfront.net/_gfx_/ic...
1,"{'severity': None, 'active': {'_rapid_links': ...","{'categoryName': 'International', 'categoryID'...","{'causeID': 20, 'causeName': 'Development and ...",Haitian Health Foundation,https://www.charitynavigator.org/?bay=search.s...,{'ratingImage': {'small': 'https://d20umu42aun...,NaN,61135999,{'deductibility': 'Contributions are deductibl...,"{'country': None, 'stateOrProvince': 'CT', 'ci...",The mission of the Haitian Health Foundation (...,"{'charityName': 'Haitian Health Foundation', '...",Changing Lives ~ One Person at a Time,http://www.haitianhealthfoundation.org/,4,{'small': 'https://d20umu42aunjpx.cloudfront.n...,https://d20umu42aunjpx.cloudfront.net/_gfx_/ic...,https://d20umu42aunjpx.cloudfront.net/_gfx_/ic...
2,"{'severity': None, 'active': {'_rapid_links': ...","{'categoryName': 'Animals', 'categoryID': 1, '...","{'causeID': 2, 'causeName': 'Animal Rights, We...",SPCA of Westchester,https://www.charitynavigator.org/?bay=search.s...,{'ratingImage': {'small': 'https://d20umu42aun...,NaN,131740069,{'deductibility': 'Contributions are deductibl...,"{'country': None, 'stateOrProvince': 'NY', 'ci...",The SPCA of Westchester was founded in 1883. T...,"{'charityName': 'SPCA of Westchester', 'ein': ...",Over 100 years of service to animals and their...,http://www.spca914.org,4,{'small': 'https://d20umu42aunjpx.cloudfront.n...,https://d20umu42aunjpx.cloudfront.net/_gfx_/ic...,https://d20umu42aunjpx.cloudfront.net/_gfx_/ic...
3,"{'severity': None, 'active': {'_rapid_links': ...","{'categoryName': 'Environment', 'categoryID': ...","{'causeID': 11, 'causeName': 'Environmental Pr...",Conservation Foundation of the Gulf Coast,https://www.charitynavigator.org/?bay=search.s...,{'ratingImage': {'small': 'https://d20umu42aun...,NaN,200345249,{'deductibility': 'Contributions are deductibl...,"{'country': None, 'stateOrProvince': 'FL', 'ci...",Conservation Foundation of the Gulf Coast work...,{'charityName': 'Conservation Foundation of th...,Conservation through education and collaboration,http://conservationfoundation.com/,4,{'small': 'https://d20umu42aunjpx.cloudfront.n...,https://d20umu42aunjpx.cloudfront.net/_gfx_/ic...,https://d20umu42aunjpx.cloudfront.net/_gfx_/ic...
4,"{'severity': None, 'active': {'_rapid_links': ...","{'categoryName': 'Community Development', 'cat...","{'causeID': 42, 'causeName': 'United Ways', 'c...",United Way of Blount County,https://www.charitynavigator.org/?bay=search.s...,{'ratingImage': {'small': 'https://d20umu42aun...,NaN,237122193,{'deductibility': 'Contributions are deductibl...,"{'country': None, 'stateOrProvince': 'TN', 'ci...",United Way of Blount County is advancing the c...,"{'charityName': 'United Way of Blount County',...",Live united,http://www.liveunitedblount.org,4,{'small': 'https://d20umu42aunjpx.cloudfront.n...,https://d20umu42aunjpx.cloudfront.net/_gfx_/ic...,https://d20umu42aunjpx.cloudfront.net/_gfx_/ic...
5,"{'severity': None, 'active': {'_rapid_links': ...","{'categoryName': 'International', 'categoryID'...","{'causeID': 20, 'causeName': 'Development and ...",Ho